In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as clr
from matplotlib import colormaps
import numpy as np
import math
import csv
import datetime

#%matplotlib qt
%matplotlib inline

In [ ]:
# Split CSV lines containing strings with quotes
def csv_split(line):
    return [l for l in csv.reader([line], quotechar='"', delimiter=",", quoting=csv.QUOTE_ALL, skipinitialspace=True)][0]

# Extract a list of workouts from Strength Log sorted by date
def parse_strengthlog_workouts(filepath):
    # Strengthlog file contains personal data and workouts split by double newlines
    strenghtlog_csv = open(filepath, "r", encoding="utf-8").read().split("\n\n")
    
    # First section contains personal info
    personal_info = [l for l in csv.reader(strenghtlog_csv.pop(0).split("\n"), quotechar='"', delimiter=",", quoting=csv.QUOTE_ALL, skipinitialspace=True)]

    # Then comes the workouts section
    labels = []
    workouts = []
    for i, workout_raw in enumerate(strenghtlog_csv):
        workout_raw = workout_raw.split("\n")
        # First workout starts with "workouts" at the top
        # Followed by the labels used in the first line every workout
        if i == 0:
            # Pop "workouts"
            workout_raw.pop(0)
            # Store labels
            labels = csv_split(workout_raw.pop(0))
        
        # Parse every workout
        # First line contains info about the workout
        workout_info = csv_split(workout_raw.pop(0))
        new_workout = dict(zip(labels, workout_info))

        # Then the excercises looks like this
        # "Övning, Bänkpress",Set,1,reps,12,weight,30
        # Convert this line to a dictionary containing name, Set, reps, weight
        excercises = []
        for excercise in workout_raw:
            excercise = csv_split(excercise)
            new_excercise = dict(zip(["name"] + excercise[1::2], excercise[::2]))
            if new_excercise:
                if new_excercise["name"] == "Instance of 'Loc'":
                    pass
                elif "Övning, " in new_excercise["name"]:
                    new_excercise["name"] = new_excercise["name"][8:]
                    excercises.append(new_excercise)
        #print(new_workout)
        new_workout["excercises"] = excercises
        workouts.append(new_workout)

    # Sort workouts so that the oldest one is first
    return sorted(workouts, key=lambda x: datetime.datetime.strptime(x["Datum"], "%Y-%m-%d"))

def print_excercise_names(workouts):
    names = set()
    for workout in workouts:
        for excercise in workout["excercises"]:
            names.add(excercise["name"])
    for n in sorted(names):
        print(n)

def extract_pbs_by_reps(workouts, name):
    # Build an array of personal bests starting from nothing
    pbs = [{"date": "1970-01-01"}]
    # Go through and create a list of bench PBs
    # "Övning, Bänkpress",Set,1,reps,12,weight,30
    # A new PB starts with the last record and only updates if it changes
    for workout in workouts:
        broke_old_pb = False
        # Based on last pb
        new_pb = dict(pbs[-1])
        new_pb["date"] = workout["Datum"]
        # Go through execises in workout and look for new pbs
        for excercise in workout["excercises"]:
            # Only look at excercise name
            if excercise["name"] != name:
                continue
            # Check reps and weight
            reps = int(excercise["reps"])
            weight = float(excercise["weight"].replace(",", "."))
            # If any weight changed, mark new entry for insertion
            # The rep number includes all values below it to not have gaps
            # E.g 2 reps at 20 kg is also counted as 1 rep best if no better exists
            for r in range(1, reps+1):
                if r not in new_pb or new_pb[r] < weight:
                    new_pb[r] = weight
                    broke_old_pb = True
        if broke_old_pb:
            pbs.append(new_pb)
    return pbs

def extract_pbs_by_weight(workouts, name, weight_step=2.5):
    # Build an array of personal bests starting from nothing
    pbs = [{"date": "1970-01-01"}]
    # Go through and create a list of bench PBs
    # "Övning, Bänkpress",Set,1,reps,12,weight,30
    # A new PB starts with the last record and only updates if it changes
    for workout in workouts:
        broke_old_pb = False
        # Based on last pb
        new_pb = dict(pbs[-1])
        new_pb["date"] = workout["Datum"]
        # Go through execises in workout and look for new pbs
        for excercise in workout["excercises"]:
            # Only look at excercise name
            if excercise["name"] != name:
                continue
            # Check reps and weight
            reps = int(excercise["reps"])
            weight = np.floor(float(excercise["weight"].replace(",", ".")) / weight_step) * weight_step
            # If any weight changed, mark new entry for insertion
            # The rep number includes all values below it to not have gaps
            # E.g 2 reps at 20 kg is also counted as 1 rep best if no better exists
            for w in np.arange(start=0.0, stop=weight+weight_step, step=weight_step):
                if w not in new_pb or new_pb[w] < reps:
                    new_pb[w] = reps
                    broke_old_pb = True
        if broke_old_pb:
            pbs.append(new_pb)
    return pbs


In [ ]:
# Calculate 1RM based on a performed set with weight and reps
def calculate_1rm(reps, weight):
    # the Brzycki formula
    return weight / (1.0278 - 0.0278 * reps)

# Calculate how many reps at a weight given 1RM and reps
def calculate_reps(weight, onerm):
    return (1.0278 - (weight / onerm)) / 0.0278

In [ ]:
def plot_by_rep(rep_pbs, max_rep, max_weight, weight_step=2.5, rm_line_step=20):
    reps = np.arange(start=1, stop=max_rep+1, step=1)
    weights = np.arange(start=0, stop=max_weight+weight_step, step=weight_step)

    # Meshgrid of X=reps and Y=weights and respective Z=1RMs
    X, Y = np.meshgrid(reps, weights)
    Z = calculate_1rm(reps=X, weight=Y)

    # Generate lines for various 1RMs
    rm_line_max = calculate_1rm(reps=max_rep, weight=max_weight)
    oneRMs = np.arange(start=rm_line_step, stop=rm_line_max, step=rm_line_step)
    
    pbs = [
        [pb.get(i, 0) for i in reps]
        for pb in rep_pbs
    ]
    
    fig, ax = plt.subplots()
    ax.set_title("PB by reps")

    # 1RM contour lines
    CS = ax.contour(X, Y, Z, levels=oneRMs, colors="black", linestyles="dashed")
    ax.clabel(CS, inline=True, fontsize=10)

    # Personal records
    p = ax.plot(reps, pbs[-1], ".", color="black")
    #for pb in pbs:
    #    ax.fill_between(reps, 0, pb, alpha=0.8/len(pbs), color="red")

    ys = []
    for i, pb in enumerate(zip([0] + pbs[:-1], pbs)):
        pba, pbb = pb
        a = 1.0
        x = i * 1.0 / len(pbs) - 0.5
        y = 0.1 + 0.9 * (np.power(x, 3) + 0.125) / 0.25
        ys.append(y)
        #c = colormaps["jet"](y)
        c = colormaps["RdYlGn"](y)
        ax.fill_between(reps, pba, pbb, alpha=a, color=c, edgecolor="black", linewidth=0)

    #m = plt.heatmap(ys, cmap='RdYlGn')
    cbar = fig.colorbar(cm.ScalarMappable(norm=clr.Normalize(0.1, 0.9), cmap="RdYlGn"),
                ax=ax, orientation="vertical")
    cbar.set_ticks([])
    cbar.ax.set_title("newest", fontsize = 8)

    ax.set_xticks(reps)
    ax.set_yticks(oneRMs)

    ax.set_xlim(1, max_rep)
    ax.set_ylim(0, max_weight)

    ax.set_xlabel("Reps")
    ax.set_ylabel("Weight", labelpad=-8.0)

    plt.show()

In [ ]:
workouts = parse_strengthlog_workouts("StrengthLog-2024-03-221.csv")
print_excercise_names(workouts)

In [ ]:
rep_pbs = extract_pbs_by_reps(workouts, "Bänkpress")
plot_by_rep(rep_pbs=rep_pbs, max_rep=20, max_weight=100)